## Toki-Pona-ChatGLM-Finetune

This notebook implements finetuning of ChatGLM on Toki Pona content to attempt to produce an LLM capable of speaking Toki Pona.

In [ ]:
import os

from transformers import AutoTokenizer, AutoModel

device = 'cpu'
checkpoint = "THUDM/chatglm-6b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True, revision = 'main')
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True, revision = 'main')
